In [1]:
from google.colab import drive
import os

#drive.mount("/content/drive")

#%cd /content/drive/MyDrive/EPITA/SCIA/ing3/DNN/

In [ ]:
!pip install transformers

In [2]:
from model import BertForTokenAndSequenceJointClassification
import torch
from transformers import BertTokenizerFast
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# Test

In [11]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [12]:
model = BertForTokenAndSequenceJointClassification.from_pretrained(
    "QCRI/PropagandaTechniquesAnalysis-en-BERT",
    revision="v0.1.0",
).to(device)

In [13]:
def classify_text_(text: str):
    inputs = tokenizer.encode_plus(text, return_tensors="pt").to(device)
    outputs = model(**inputs)

    token_class_index = torch.argmax(outputs.token_logits, dim=-1)
    tags = [model.token_tags[i] for i in token_class_index[0].tolist()[1:-1]]

    sequence_class_index = torch.argmax(outputs.sequence_logits, dim=-1)
    sequence_class = model.sequence_tags[sequence_class_index[0]]
    return sequence_class

In [19]:
classify_text_('Richard Dawkins, an evolutionary biologist and perhaps the foremost expert in the field, says that evolution is true. Therefore, it is true.')

'Non-prop'

In [20]:
PROMPT = "a close up of a sign advertising a front national restaurant"
inputs = tokenizer.encode_plus(PROMPT, return_tensors="pt")
inputs

{'input_ids': tensor([[ 101,  170, 1601, 1146, 1104,  170, 2951, 6437,  170, 1524, 1569, 4382,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [21]:
outputs = model(**inputs)
#outputs

In [22]:
sequence_class_index = torch.argmax(outputs.sequence_logits, dim=-1)
sequence_class_index

tensor([0])

In [23]:
sequence_class = model.sequence_tags[sequence_class_index[0]]
sequence_class

'Non-prop'

In [24]:
token_class_index = torch.argmax(outputs.token_logits, dim=-1)
token_class_index

tensor([[7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [26]:
tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0][1:-1])
#tokens

In [27]:
tags = [model.token_tags[i] for i in token_class_index[0].tolist()[1:-1]]
tags

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# Handle the dataset

In [ ]:
# ONLY IF WE WANT TO MODIFY CURRENT DATA
df1 = pd.read_csv('img2text_propaganda.csv')
df2 = pd.read_csv('img2text_movie_propaganda.csv')

# Concatenate the DataFrames
merged_df = pd.concat([df1, df2], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_csv_path = 'img2text_compare.csv'
merged_df.to_csv(merged_csv_path, index=False)

print(f"The two CSV files have been merged into {merged_csv_path}")

The two CSV files have been merged into img2text_compare.csv


In [5]:
df = pd.read_csv('img2text_compare (2).csv')
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df.head()

,title,classes,generated_text_1,generated_text_2
0,24968.jpg,none_propaganda,a photography of a poster for charlie chan london,a poster for charlie chan london starring in t...
1,image_389.png,propaganda,a photography of a poster advertising a politi...,a black and white poster with a black border a...
2,image_196.png,propaganda,a photography of a poster advertising a concer...,a poster advertising a concert at the grand ball
3,9893.jpg,none_propaganda,a photography of a woman in a dress and hat wi...,a poster for a concert with a woman in a dress...
4,image_357.png,propaganda,a photography of a poster advertising a french...,a poster advertising the birth of the french r...


# Classify

In [6]:
def classify_text_(text: str, model, tokenizer, device):
    inputs = tokenizer.encode_plus(text, return_tensors="pt").to(device)
    outputs = model(**inputs)

    token_class_index = torch.argmax(outputs.token_logits, dim=-1)
    tags = [model.token_tags[i] for i in token_class_index[0].tolist()[1:-1]]

    sequence_class_index = torch.argmax(outputs.sequence_logits, dim=-1)
    sequence_class = model.sequence_tags[sequence_class_index[0]]
    return sequence_class

In [9]:
def predict_best_classes(df, model, tokenizer, device):
    # Select the relevant columns
    texts = df['generated_text_2'].tolist()
    true_labels = df['classes'].tolist()

    # Create an empty list to store predicted labels
    predicted_labels = []

    # Initialize variables for accuracy calculation
    correct_predictions = 0
    correct_predictions_prop = 0
    total_samples = len(texts)

    # Evaluate each text and store the predicted label
    for text, true_label in zip(texts, true_labels):
      predicted_label = classify_text_(text, model, tokenizer, device)
      if predicted_label == 'Prop':
        predicted_label = 'propaganda'
        if predicted_label == true_label:
          correct_predictions_prop += 1
        print(f"text is: {text}")# and true_label is : {true_label} and predicted is: {predicted_label}")
      else:
        predicted_label = 'none_propaganda'
      predicted_labels.append(predicted_label)

      # Check accuracy
      if predicted_label == true_label:
        correct_predictions += 1
    # Calculate accuracy
    accuracy = (correct_predictions / total_samples) * 100

    # Add predicted labels to the DataFrame
    df['predicted_class'] = predicted_labels
    propaganda_count = df['predicted_class'].value_counts().get('propaganda', 0)
    accuracy_prop = (correct_predictions_prop / propaganda_count)* 100


    return accuracy, accuracy_prop

### Prediction results

In [10]:
# Tokenize the texts
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Instantiate your BERT model
model = BertForTokenAndSequenceJointClassification.from_pretrained(
    "QCRI/PropagandaTechniquesAnalysis-en-BERT",
    revision="v0.1.0",
).to(device)

# Predict the best classes for each text and calculate accuracy
accuracy, accuracy_prop = predict_best_classes(df, model, tokenizer, device)

print(f'Accuracy: {accuracy:.2f}% and accuracy of propaganda prediction is: {accuracy_prop:.2f}%')

text is: a poster for the vampire bat starring two women and a man
text is: a poster for the man who knew too much
text is: a poster for the movie she done him wrong
text is: a poster for the film cinderella starring mary pickford and cinderella
text is: a poster for the movie harold lloyd and the cats paw
text is: a poster for the movie crazy lady
text is: a poster for the film the death kiss
text is: a poster for the old dark house
text is: a poster for the movie the mayor of hell
text is: a poster for the film hallein'in a rum
text is: a poster for the movie kentucky kernels
text is: a poster for the movie the art of the howling dog
text is: richard the knight - 11 x 17 movie poster - style a
text is: they fight for you protect them poster
text is: a picture of a poster for the movie arbuckle and keaton
text is: there is a poster of a flower with a halo on it
text is: the devil's brother - 11 x 17 movie poster - style a
text is: a picture of a movie poster for the movie sweet little